# Enriching the Visier Model
## Overview
This is a Jupyter Notebook sample that shows how to enrich the data in an analytic object which is primarily loaded from other sources.
This sample in of itself has no intrinsic business value. Instead, its purpose is exclusively to show how to call various Visier Public APIs to query for source data, combine it with an external data set and finally, to write back the results.

Specifically, we will be combining Visier Employee data with [publicly available name data](https://www.ssa.gov/oact/babynames/limits.html) from the US Government Social Security Administration for the purposes of populating a new `Employee` property called `Name_Rank`.

### Prerequisites
In order to run this example, the following prerequisites must be satisfied:
1. Create and publish to production a simple, numeric property named `Name_Rank` on the `Employee` object
1. Define and assign a profile that enables writing data and reading model metadata through APIs. In addition the following Capabilities are needed:
   1. Direct Publish
   1. Legacy API Access
   1. Manage Jobs
   1. Upload Data
1. Defined an `.env` file as outlined in the [Python Connector documentation](https://github.com/visier/connector-python#jupyter-basic-authentication-example)
1. Downloaded the [National Data names file](https://www.ssa.gov/oact/babynames/names.zip) from SSA and extracted file `yob2022.txt` and copied it into a directory named `data`.

## Load Enrichment Data
We use the ubiquitous `pandas` library both to represent data sets through `Dataframe`s as well as for file handling. The source files are comma-separated so we load it through the `read_csv` method.
The file does not contain a header row, so we provide a header with names that will align with names of Visier properties. We perform very simple transformation where we sort the results by occurrence of each name and assign the rank based on row number of the sorted lists.
We also define an index to facilitate a subsequent join-operation with Visier `Employee` data.

As we will be simplifying the gender designations on the Visier side to match the SSA data, we create two distinct SSA name data sets: `f_names` and `m_names`

In [54]:
import pandas as pd

def filter_and_index(df, gender):
    df.loc[df['Gender'] == gender]
    df['Name_Rank'] = df.reset_index().index + 1
    return df.set_index(['First_Name', 'Gender'])

names = pd.read_csv('data/yob2022.txt', header=0, names=['First_Name', 'Gender', 'Count'])
names.sort_values(inplace=True, by='Count', ascending=False)
f_names = filter_and_index(names, 'F')
m_names = filter_and_index(names, 'M')

## Query for Employee Data
At this point, we are ready to instantiate the Visier Python Connector. We read credentials, build an `auth` object which we use to login to Visier and get a session object, `s`.
Because we need to execute a Visier SQL-like query to get the list of `Employee`s whose name we wish to rank in this sample, we import the `QueryApiClient`.

Gender designation for the Visier `Employee` object is completely customizable and in order to ensure we can align with the SSA-sourced data, we're going to execute two distinct but similar SQL-like queries; where `isFemale` is `TRUE` and when it is `FALSE`. Once we have these two distinct data sets, we manually ascribe the identical gender keys that is present in the SSA data set.

In [55]:
import re
from dotenv import dotenv_values
from visier.connector import VisierSession, make_auth
from visier.api import QueryApiClient

env_creds=dotenv_values()
auth = make_auth(env_values=env_creds)

def mk_df(client, query):
    """Run a list query and return the results as a Pandas DataFrame.
    Transforms the result by ensuring the First_Name column contains
    single words"""
    result = client.sqllike(query)
    df = pd.DataFrame.from_records(data=result.rows(), columns=result.header)
    df['First_Name'] = df['First_Name'].transform(lambda x: re.split(r'[ -]', x)[0])
    return df

    
with VisierSession(auth) as s:
    query_client = QueryApiClient(s)
    [f_df, m_df] = [mk_df(query_client, f"""SELECT EmployeeID,
                                                   First_Name
                                            FROM Employee
                                            WHERE isFemale={is_female} AND Visier_Time BETWEEN date('2021-01-01') AND date('2022-01-01')""") for is_female in ["TRUE", "FALSE"]]
    f_df['Gender'] = 'F'
    m_df['Gender'] = 'M'


## Prepare the Data Set to upload
We now have two distinct data sets, the Visier `Employee` data and the SSA name ranking data and it is time to bring them together.
Handling the two gender designations independently, we join the Visier data with the SSA data set after ensuring the Visier data set has been suitably indexed to enable an `inner` join at the correct level of granularity.

### EventDate
Once we have concatenated the two data sets, we have to perform one more transformation on the result: We have to add a column named `EventDate` and give it a date string value formatted like `yyyy-MM-dd`. We choose the value of 'yesterday' here. That we picked 'yesterday' is not important. What is important is to understand how this value impacts you `Employee` object's data. The employees whose first names were found in the SSA data set, will have their current state record ended and immediately followed by a new record that reflects the change.

### Filename
Once the Dataframe has been updated, we save it to a file that will uploaded to Visier in the next step. In this case, where we simply upload the file as is, the name of the file itself is not important. However, should the data set be large enough to merit compression, such as through a `.zip` archive, the name of the file _inside_ the archive matters. In this case, because we are augmenting the data of an object which is primarily loaded through other means (after all, we're just adding a name rank to the `Employee`), we would have to name the file after the target object name followed by the suffix `DDIExt`. That's why the file is named `EmployeeDDIExt.csv` so that it could be added to a compressed archive and still work.
If, the other hand, we were loading objects directly, that is the Direct Intake API is used as the main source for data, the name of the file _inside_ the archive should match the name of the target object, for example `Employee.csv`,

In [56]:
import datetime

def do_join(visier_df, ssa_df):
    result=visier_df.set_index(['First_Name', 'Gender']).join(ssa_df, how='inner')
    return result.reset_index()[['EmployeeID', 'Name_Rank']]

# Build each gender result and combine
f_result=do_join(f_df, f_names)
m_result=do_join(m_df, m_names)
result=pd.concat([f_result, m_result])

# Data Set requires column `EventDate` with the date of the change
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
yesterday_str = yesterday.strftime('%Y-%m-%d')
result['EventDate'] = yesterday_str

# Write to file to be uploaded below
result_filename='EmployeeDDIExt.csv'
result.to_csv(result_filename, index=False)

## Upload the Data Set using the Direct Intake API
When uploading data, we again use the Python connector. It is instantiated and initialized the same way as we did above in order to query for Visier data.

### Direct Intake API
The differences start with instantiating a different API client, namely the `DirectIntakeApiClient`.

This specific sample is targeted to customers who use other means of loading the bulk of their data, such as uploading source files via SFTP which will be processed by the Visier Data Provisioning engine. Because we're using the Direct Intake API in a data supplementary fashion in general and augmenting `Employee` in particular, we have to provide `Configuration` that reflects that.

### Upload Transaction
Direct Intake API offers a transactional interface. This allows callers to upload many files before committing them to the system. In this case, we only have one data file, the we need three calls to upload and process one file:
* `start_transaction` to begin a transaction. We have to retain the transaction ID returned from this function.
* `upload_file` lets us point to a target object and provide a file with data that matches the appropriate sub set of properties that exist on said object.
* `commit_transaction` closes the transaction to additional upload requests and intiates the processing.

In [ ]:
from visier.api import DirectIntakeApiClient
from visier.api.direct_intake import Configuration

with VisierSession(auth) as s:
    intake_client = DirectIntakeApiClient(s, raise_on_error=True)

    # Configure the Direct Intake to supplement data in the tenant
    # Enable loading into Employee using extension tables
    config = Configuration(is_supplemental=True,
                           extend_objects=['Employee'])
    returned_config = intake_client.set_configuration(config)
    
    # Upload the file within the context of a transaction
    try:
        tx_response = intake_client.start_transaction().json()
        print(tx_response)
        transaction_id = tx_response['transactionId']
        intake_client.upload_file(transaction_id, 'Employee', result_filename)
        intake_client.commit_transaction(transaction_id)
        print(f'Committed {transaction_id}')
    except Exception as ex:
        print(f'Rolling back {transaction_id}', ex)
        intake_client.rollback_transaction(transaction_id)

Once the provisioning is complete, the data is available to query.